# Ayudantía 1 - Notebook 2
### Profesor: Elwin van 't Wout
### Ayudante: Alberto Almuna Morales (alberto.almuna@uc.cl)

The library ```joblib``` provides functionality for parallel computing. In this notebook, let us use a parallel pool of workers for different tasks.

In [1]:
import numpy as np

The `Parallel` class of `joblib` creates a pool of workers to which tasks can be assigned. This pool of workers can be reused for different sets of tasks.

In [2]:
from joblib import Parallel, delayed

Let us create a pool with the maximum number of workers available on our machine. Specifying the number of jobs as minus one means the maximum number of workers that can automatically be found on the machine.

In [3]:
from joblib import cpu_count
print("Number of cores found by joblib:", cpu_count())

Number of cores found by joblib: 8


In [4]:
parallel_pool = Parallel(n_jobs=-1)

Let us create two different functions we like to perform: taking the square and the square root. For the square root, we can use the `Numpy` function, but for the square we create our own function.

In [5]:
def my_square(n):
    return n**2
parallel_square = delayed(my_square)
parallel_root = delayed(np.sqrt)

Creating the tasks requires specifying the input variables. For the square, let us use a uniform sample for values between zero and one. For the square root, we'd like to use the previous output and check if the result is the input again. Hence, we first need to perform the square operations.

In [8]:
input_values = np.linspace(0,1,10)
print("Input values are:", input_values)

Input values are: [0.         0.11111111 0.22222222 0.33333333 0.44444444 0.55555556
 0.66666667 0.77777778 0.88888889 1.        ]


In [9]:
parallel_tasks_square = [parallel_square(i) for i in input_values]

In [10]:
parallel_results_square = parallel_pool(parallel_tasks_square)

In [11]:
print("The squares of input values are:", parallel_results_square)

The squares of input values are: [0.0, 0.012345679012345678, 0.04938271604938271, 0.1111111111111111, 0.19753086419753085, 0.308641975308642, 0.4444444444444444, 0.6049382716049381, 0.7901234567901234, 1.0]


In [12]:
parallel_tasks_root = [parallel_root(i) for i in parallel_results_square]

In [13]:
parallel_results_root = parallel_pool(parallel_tasks_root)

In [14]:
print("The square roots of the squares of the input values are:", parallel_results_root)

The square roots of the squares of the input values are: [0.0, 0.1111111111111111, 0.2222222222222222, 0.3333333333333333, 0.4444444444444444, 0.5555555555555556, 0.6666666666666666, 0.7777777777777777, 0.8888888888888888, 1.0]


Notice that the square root of the square of the input values are indeed the input variables, perhaps with a small rounding error. Also, the same pool of workers was used twice: the tasks needed to be defined again but the same worker pool can be used many times. Reusing the same worker pool tends to be quicker since it can be initialized once and applied to different tasks.